In [9]:
import logging
from google.auth import credentials
from google.auth.transport.requests import Request
from google.cloud import bigquery
import json
import os
import numpy as np
import pandas as pd

# Define your Google Cloud project ID directly in the notebook
project_id = "logistics-data-storage-staging"  # Set your project ID here directly

logging.info(f"Initializing BigQuery client for project: {project_id}")

# Load the credentials from the file created in the GitHub Action workflow
credentials_path = '/tmp/credentials.json'

# Check if the credentials file exists
if os.path.exists(credentials_path):
    # Load credentials from the file
    with open(credentials_path, 'r') as f:
        creds_data = json.load(f)
    
    # Rebuild credentials object using the loaded data
    credentials = credentials.Credentials.from_authorized_user_info(creds_data)
    
    # Refresh credentials if necessary
    if credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
else:
    raise FileNotFoundError(f"Credentials file not found at {credentials_path}")

# Now initialize the BigQuery client with the credentials and project_id explicitly set
try:
    client = bigquery.Client(credentials=credentials, project=project_id)
    logging.info(f"BigQuery client initialized successfully for project: {project_id}")
except Exception as e:
    logging.error(f"Failed to initialize BigQuery client: {e}")
    raise e

# Define the query
test_query = """
Select *
FROM `logistics-data-storage-staging.long_term_pricing.abc_performance` 
"""

# Run the query and store the result in a DataFrame
test_df = client.query(test_query).to_dataframe()

# Display the result
test_df.head()  # You can display the first few rows of the result
